<h2>IITI MNIST from CSV, skb</h2>

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import tensorflow as tf
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split


<h3>LOADING training andd test csvs</h3>

In [ ]:
data_train = pd.read_csv("data/train.csv")
data_test = pd.read_csv("data/test.csv")
print(data_test.shape)
data_test.head()

In [ ]:
#Training images and lables: X and Y

X = np.array(data_train.iloc[:,1:]) #data
Y = np.array(data_train.iloc[:,0]) #labels

In [ ]:
Y = to_categorical(Y,10)
Y

In [ ]:
#splitting data into training and sub-training 80 , validation = 20
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=13)

In [ ]:
# X_test forms the test images, and y_test forms the test labels
#X_test = np.array(data_test.iloc[:, 1:])
#In my dataset for test there was no Y labels and no data to be filtered like avoid columns etc
X_test = data_test.values.astype("float32")
X_test.shape

In [ ]:
#dimension of inputs be 28x28x1 grayscale
input_shape = (28,28,1)


<h3>images are to be reshaped  and scaled to have values between 0 and 1 range </h3>

In [ ]:
#Preparing the training images.
X_train = X_train.reshape(X_train.shape[0],28,28,1)
X_train = X_train.astype("float32")
X_train = X_train/255


In [ ]:
#Preparing the test images
X_test = X_test.reshape(X_test.shape[0],28,28,1)
X_test = X_test/255
#X_test = X_test.reshape(X_test.shape[0],28,28,1)

In [ ]:
#Preparing the val images
X_val = X_val.reshape(X_val.shape[0],28,28,1)
X_val = X_val.astype("float32")
X_val = X_val/255

In [ ]:
print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_val.shape[0], 'validation samples')
print(X_test.shape[0], 'test samples')

<h3>Making the model and training it

In [ ]:
classifier = Sequential()

classifier.add(Conv2D(32,(3,3),activation="relu",input_shape=input_shape))
classifier.add(MaxPooling2D(2,2))
classifier.add(Conv2D(32,(3,3),activation="relu"))
classifier.add(MaxPooling2D(2,2))

classifier.add(Flatten())

classifier.add(Dense(128,activation="relu"))
classifier.add(Dense(10,activation = "softmax"))

classifier.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
classifier.summary()

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=15, # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


In [ ]:
datagen.fit(X_train)

In [ ]:
classifier.fit_generator(datagen.flow(X_train,y_train,batch_size=32),
                         epochs=20,
                         validation_data=(X_val,y_val),
                         steps_per_epoch=1000
                        )

In [ ]:
classifier.evaluate(X_val,y_val)

In [ ]:
classifier.save("digit_skb_iiti.h5")

In [ ]:
from keras.preprocessing import image
import cv2 as cv
import os
#loading my test image
'''
#non_gray = cv.imread("handwritten.jpg")
gray = cv.imread("handwritten.jpg" , cv.IMREAD_GRAYSCALE)
#cv.imshow("image",gray)
cv.imwrite("handwrittens.jpg",gray)
'''
hand_Written = cv.imread("gray.png" , cv.IMREAD_GRAYSCALE)
hand_Written = cv.resize(hand_Written,(28,28))
#hand_Written = image.load_img("handwrittens.jpg", target_size=(28,28))
hand_Written = image.img_to_array(hand_Written)
print(hand_Written.shape)
hand_Written = np.expand_dims(hand_Written,axis=0)

In [ ]:
result = classifier.predict_classes(hand_Written)
result